This notebook contains code for rendering .png images according to a predefined fsleyes orthographic view, of 3D-FLAIR based MNI normalized nrCBV maps overlayed on a MNI brain regions template. It also uses ffmpeg to make videos of the images.

1. Specify epi_corrections output directory

In [3]:
output_directory_suffix = "2019_07_02"
# On local file system:
corrections_base_directory = "../../epi_corrections_out_" + output_directory_suffix
# On samba share (remote file sytem):
#corrections_base_directory = "/run/user/1000/gvfs/smb-share:server=192.168.1.207,share=hdd3tb1/data/IVS_EPI_BASELINE/epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../../epi_corrections_out_2019_07_02'

2. Since nICE introduced files with spaces in their file names, 
recursively replace spaces in folders and file names.
The fsleyes rendering bash command did not accept 
spaces in file names and directories. 

In [4]:
from utils import replace_spaces_with_underscore
replace_spaces_with_underscore(corrections_base_directory + "/EPI_raw_DSC")
replace_spaces_with_underscore(corrections_base_directory + "/EPI_applytopup")
replace_spaces_with_underscore(corrections_base_directory + "/EPI_applyepic")

3. Find the MNI-normalized, NordicICE intensity Normalized Relative CBV files. Can be done remotely.

In [5]:
from search import find_cbv_files

[raw_cbv_files_e1, \
 topup_cbv_files_e1, \
 epic_cbv_files_e1, \
 raw_cbv_files_e2, \
 topup_cbv_files_e2, \
 epic_cbv_files_e2] = find_cbv_files(corrections_base_directory)

In [6]:
# A check
print("Equal number of detected CBV files for raw (uncorrected), topup, and epic correction methods: %r" % \
      (len(raw_cbv_files_e1) == \
       len(raw_cbv_files_e2) == \
       len(topup_cbv_files_e1) == \
       len(topup_cbv_files_e2) == \
       len(epic_cbv_files_e1) == \
       len(epic_cbv_files_e2)))
print("Number of subject CBVs: %i" % len(raw_cbv_files_e1))

Equal number of detected CBV files for raw (uncorrected), topup, and epic correction methods: True
Number of subject CBVs: 45


4. Find the corresponding MNI label files 
(should perhaps be identical to each other, but just to make sure..)

In [7]:
from search import find_label_files

[raw_label_files_e1, \
 topup_label_files_e1, \
 epic_label_files_e1, \
 raw_label_files_e2, \
 topup_label_files_e2, \
 epic_label_files_e2] = find_label_files(corrections_base_directory)

In [8]:
# A check
print("Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: %r" % \
      (len(raw_label_files_e1) == \
       len(raw_label_files_e2) == \
       len(topup_label_files_e1) == \
       len(topup_label_files_e2) == \
       len(epic_label_files_e1) == \
       len(epic_label_files_e2)))
print("Number of subject label files: %i" % len(raw_label_files_e1))

Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: True
Number of subject label files: 45


5. For the raw, topup and epic Gradient Echo (GE) and Spin Echo (SE) nrCBV data + MNI regions file, render .png images in a predefined orthographic view, then make a video. __Note:__ The following cells must run within a fsl X environment

In [10]:
%%bash -s "{" ".join(raw_cbv_files_e1)}" "{" ".join(topup_cbv_files_e1)}" "{" ".join(epic_cbv_files_e1)}" "{" ".join(raw_cbv_files_e2)}" "{" ".join(topup_cbv_files_e2)}" "{" ".join(epic_cbv_files_e2)}" "{" ".join(raw_label_files_e1)}" "{" ".join(topup_label_files_e1)}" "{" ".join(epic_label_files_e1)}" "{" ".join(raw_label_files_e2)}" "{" ".join(topup_label_files_e2)}" "{" ".join(epic_label_files_e2)}"
IFS=' ' read -r -a raw_cbv_files_e1 <<< "$1"
IFS=' ' read -r -a topup_cbv_files_e1 <<< "$2"
IFS=' ' read -r -a epic_cbv_files_e1 <<< "$3"
IFS=' ' read -r -a raw_cbv_files_e2 <<< "$4"
IFS=' ' read -r -a topup_cbv_files_e2 <<< "$5"
IFS=' ' read -r -a epic_cbv_files_e2 <<< "$6"
IFS=' ' read -r -a raw_label_files_e1 <<< "$7"
IFS=' ' read -r -a topup_label_files_e1 <<< "$8"
IFS=' ' read -r -a epic_label_files_e1 <<< "$9"
IFS=' ' read -r -a raw_label_files_e2 <<< "${10}"
IFS=' ' read -r -a topup_label_files_e2 <<< "${11}"
IFS=' ' read -r -a epic_label_files_e2 <<< "${12}"
render() {
    local -n _cbv_files=$1
    local -n _label_files=$2
    local -n _render_dir=$3
    
    # Remove existing render directory.
    if [ -d "$_render_dir" ]; then rm -rd "$_render_dir"; fi
    
    mkdir -p "$_render_dir"
    
    for index in "${!_cbv_files[@]}"
    do
        cbv_file="${_cbv_files[index]}"
        label_file="${_label_files[index]}"
        
        cbv_file_name="$(basename $cbv_file)"
        label_file_name="$(basename $label_file)"
        
        render_command='fsleyes 
            render 
            --outfile "'$_render_dir'/'output_$(printf "%03d\n" $index)'" 
            --scene ortho 
            --worldLoc 9.918212890625e-05 -18.000099182128906 7.999900817871094 
            --displaySpace 
            '$cbv_file' 
            --xcentre -0.00000 -0.11019 
            --ycentre -0.00000 -0.11019 
            --zcentre -0.00000 -0.00000 
            --xzoom 803.0573770888577 
            --yzoom 809.4712464952298 
            --zzoom 803.057377088858 
            --hideLabels 
            --labelSize 14 
            --layout horizontal 
            --hideCursor 
            --bgColour 0.0 0.0 0.0 
            --fgColour 1.0 1.0 1.0 
            --cursorColour 0.0 1.0 0.0 
            --colourBarLocation top 
            --colourBarLabelSide top-left 
            --performance 3 
            '$cbv_file' 
            --name '$cbv_file_name' 
            --overlayType volume 
            --alpha 100.0 
            --brightness 64.94720130619707 
            --contrast 79.89440261239416 
            --cmap hot 
            --negativeCmap greyscale 
            --displayRange 0.0 12.0 
            --clippingRange 0.0 30.140860195159913 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0 
            '$label_file' 
            --name '$label_file_name' 
            --overlayType volume 
            --alpha 37.33333333798995 
            --brightness 50.0 
            --contrast 50.0 
            --cmap greyscale 
            --negativeCmap greyscale 
            --displayRange 0.0 207.0 
            --clippingRange 0.0 209.07 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0'
        echo $render_command >> "$_render_dir/commands.txt"
        #echo $render_command
        eval $render_command
        echo "--Finished rendering volume $index to .png--"
    done
    ffmpeg -framerate 1 -i "$_render_dir"/output_%03d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -preset slow -profile:v high -level:v 4.0 -pix_fmt yuv420p -crf 1 "$_render_dir"/video.mp4
    echo "--Finished rendering pngs to video--"
}
render_dir_raw_e1="/home/ivar/Desktop/temp/render/EPI_raw_DSC/e1"
render raw_cbv_files_e1 raw_label_files_e1 render_dir_raw_e1
sleep 1 # Sleep 1 second; this was necessary otherwise the next call of render() function did seem to get emtpy render dir argument...
render_dir_raw_e2="/home/ivar/Desktop/temp/render/EPI_raw_DSC/e2"
render raw_cbv_files_e2 raw_label_files_e2 render_dir_raw_e2
sleep 1
render_dir_topup_e1="/home/ivar/Desktop/temp/render/EPI_applytopup/e1"
render topup_cbv_files_e1 topup_label_files_e1 render_dir_topup_e1
sleep 1
render_dir_topup_e2="/home/ivar/Desktop/temp/render/EPI_applytopup/e2"
render topup_cbv_files_e2 topup_label_files_e2 render_dir_topup_e2
sleep 1
render_dir_epic_e1="/home/ivar/Desktop/temp/render/EPI_applyepic/e1"
render epic_cbv_files_e1 epic_label_files_e1 render_dir_epic_e1
sleep 1
render_dir_epic_e2="/home/ivar/Desktop/temp/render/EPI_applyepic/e2"
render epic_cbv_files_e2 epic_label_files_e2 render_dir_epic_e2

--Finished rendering volume 0 to .png--
--Finished rendering volume 1 to .png--
--Finished rendering volume 2 to .png--
--Finished rendering volume 3 to .png--
--Finished rendering volume 4 to .png--
--Finished rendering volume 5 to .png--
--Finished rendering volume 6 to .png--
--Finished rendering volume 7 to .png--
--Finished rendering volume 8 to .png--
--Finished rendering volume 9 to .png--
--Finished rendering volume 10 to .png--
--Finished rendering volume 11 to .png--
--Finished rendering volume 12 to .png--
--Finished rendering volume 13 to .png--
--Finished rendering volume 14 to .png--
--Finished rendering volume 15 to .png--
--Finished rendering volume 16 to .png--
--Finished rendering volume 17 to .png--
--Finished rendering volume 18 to .png--
--Finished rendering volume 19 to .png--
--Finished rendering volume 20 to .png--
--Finished rendering volume 21 to .png--
--Finished rendering volume 22 to .png--
--Finished rendering volume 23 to .png--
--Finished rendering volum

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena